In [26]:
import pandas as pd
from sklearn import model_selection, metrics

import plotly.express as px
from datetime import timedelta

In [32]:
data = pd.read_csv("../data/MSFT.csv")
data['Date'] = pd.to_datetime(data['Date'])

In [35]:
data = data.set_index("Date")

In [37]:
last_date = data.index[-1]

# Create a 2-week offset
offset = timedelta(weeks=2)

# Get the test set
test_set = data[last_date-offset:]

# Get the train set
train_set = data[:last_date-offset]

In [41]:
X_train = train_set[["Open", "High", "Low"]]
y_train = train_set["Close"]
y_train = y_train.values.reshape(-1, 1)

X_test = test_set[["Open", "High", "Low"]]
y_test = test_set["Close"]
y_test = y_test.values.reshape(-1, 1)


In [42]:
X_train

,Open,High,Low
Date,,,
2009-05-07,19.959999,19.990000,19.139999
2009-05-08,19.459999,19.639999,19.139999
2009-05-11,19.200001,19.730000,19.010000
2009-05-12,19.510000,20.059999,19.469999
2009-05-13,19.920000,20.000000,19.670000
...,...,...,...
2022-12-23,236.110001,238.869995,233.940002
2022-12-27,238.699997,238.929993,235.830002
2022-12-28,236.889999,239.720001,234.169998


In [50]:
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM

model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (X_train.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(32),)
model.add(Dense(1))
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 3, 128)            66560     
                                                                 
 lstm_9 (LSTM)               (None, 64)                49408     
                                                                 
 dense_8 (Dense)             (None, 32)                2080      
                                                                 
 dense_9 (Dense)             (None, 1)                 33        
                                                                 
Total params: 118,081
Trainable params: 118,081
Non-trainable params: 0
_________________________________________________________________


In [46]:
from keras.callbacks import EarlyStopping
from keras.metrics import RootMeanSquaredError

model.compile(optimizer='adam', loss='mean_squared_error', metrics=[RootMeanSquaredError()])
model.fit(X_train, y_train, batch_size=32, epochs=30, validation_split=0.3,
callbacks=EarlyStopping('root_mean_squared_error', 1e-6, 5, 1))

Epoch 1/30
76/76 [==============================] - 6s 25ms/step - loss: 2.1748 - root_mean_squared_error: 1.4747 - val_loss: 4425.0757 - val_root_mean_squared_error: 66.5212
Epoch 2/30
76/76 [==============================] - 1s 12ms/step - loss: 0.2415 - root_mean_squared_error: 0.4914 - val_loss: 4418.7847 - val_root_mean_squared_error: 66.4739
Epoch 3/30
76/76 [==============================] - 1s 12ms/step - loss: 0.2550 - root_mean_squared_error: 0.5050 - val_loss: 4411.6973 - val_root_mean_squared_error: 66.4206
Epoch 4/30
76/76 [==============================] - 1s 12ms/step - loss: 0.2649 - root_mean_squared_error: 0.5147 - val_loss: 4427.3042 - val_root_mean_squared_error: 66.5380
Epoch 5/30
76/76 [==============================] - 1s 12ms/step - loss: 0.3231 - root_mean_squared_error: 0.5684 - val_loss: 4436.7129 - val_root_mean_squared_error: 66.6087
Epoch 6/30
76/76 [==============================] - 1s 12ms/step - loss: 0.3667 - root_mean_squared_error: 0.6056 - val_loss:

In [47]:
# validade model

test_loss, test_rmse = model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 40ms/step - loss: 2450.6895 - root_mean_squared_error: 49.5044


In [23]:
# test_model

import numpy as np
#features = [Open, High, Low]
features = np.array([[177.089996, 180.419998, 177.070007]])
model.predict(features)

1/1 [==============================] - 1s 840ms/step


array([[177.5495]], dtype=float32)

In [ ]:
# plot test and prediction

import numpy as np
import plotly.express as px
import plotly.graph_objects as go


fig = go.Figure([
    go.Scatter(x=X_train.squeeze(), y=y_train, name='train', mode='markers'),
    go.Scatter(x=X_test.squeeze(), y=y_test, name='test', mode='markers'),
    go.Scatter(x=x_range, y=y_range, name='prediction')
])
fig.show()